In [3]:
from src.daily_notes_processor import DailyNotesProcessor
from src.config import Config
from src.audio_processor import AudioProcessor
from src.note_generator import NoteGenerator
from src.timeline_generator import TimelineGenerator
from src.todo_extractor import TodoExtractor
from src.todo_manager import TodoManager

from datetime import datetime
from faster_whisper import WhisperModel, BatchedInferencePipeline

## Step by Step

In [4]:
config = Config('config.yaml')
config_gpt = Config('config.yaml')
config_deep = Config('config_debug.yaml')
audio_processor = AudioProcessor(config_gpt)

# No need to pass API keys - they're handled in the constructors
note_generator_gpt = NoteGenerator(
    config_gpt,
    model=config_gpt.model,
    temperature=config_gpt.temperature
)

note_generator_deep = NoteGenerator(
    config_deep,
    model=config_deep.model,
    temperature=config_deep.temperature
)


# Initialize TodoExtractor
todo_extractor_gpt = TodoExtractor(
    config_gpt,
    note_generator_gpt,
    audio_processor
)

todo_extractor_deep = TodoExtractor(
    config_deep,
    note_generator_deep,
    audio_processor
)

todo_manager_gpt = TodoManager(config_gpt, temperature=config_gpt.temperature)
todo_manager_deep = TodoManager(config_deep, temperature=config_deep.temperature)

audio_processor = AudioProcessor(config_gpt)

In [5]:
supported_formats = config.config_data['audio']['supported_formats']
audio_files = []

for format_ext in supported_formats:
    audio_files.extend(config.audio_input_path.glob(f"*{format_ext}"))
    
sorted(audio_files)
audio_files

[WindowsPath('c:/Users/nicco/OneDrive/Documenti/Obsidian/daily_notes/AudioInbox/Daily_Log_25-06-2025.mp3')]

In [6]:
audio_path = audio_files[0]

print(f"\n{'='*50}")
print(f"Processing: {audio_path.name}")
print(f"{'='*50}")

# Extract date from filename if possible
date_str = todo_extractor_gpt.extract_date_from_filename(audio_path.name)
if date_str:
    print(f"Date extracted from filename: {date_str}")
else:
    date_str = datetime.now().strftime('%Y-%m-%d')
    print(f"Using current date: {date_str}")

# 1. Transcribe audio
# transcript_data = audio_processor.transcribe(audio_path)
transcript_data = { 'text' :" so today I worked on my main project so silencing and apart from that I continued working with the plotting function the plotting functions for plotting and inspecting data from diffusion runs I make it work I made it work with how the data are saved  I return in the function to sample from the diffusion model so it returns the sampled sequence and dictionary of data this dictionary has several entries and each entry is a stack tensor of probabilities for  all the steps rather than a list of tens of probability for each step and I had to make the functions work with this and now they are working I also had some smaller features like printing metadata when plotting the interactive plot that's working I made some more small changes into the script  I made some more small changes into into the script for visualizing and for saving datas and I prepared everything for the PR I made the PR then I also reviewed the PR that Ishan made  Then I also continued working on updating the conference post, writing down some comments on some plots I showed last Thursday. Nothing more than that."}
print(f"Transcription completed ({len(transcript_data['text'])} chars)")

# 2. Get available projects
available_projects = config.get_available_projects()
# print(f"Available projects: {', '.join(available_projects)}")



Processing: Daily_Log_25-06-2025.mp3
Date extracted from filename: 2025-06-25
Transcription completed (1094 chars)


In [7]:
audio_filename=audio_files[0].name
output_path=config.daily_notes_path

# Generate content from transcript
content_gpt, response_gpt   = note_generator_gpt.generate_note_content(transcript_data['text'], available_projects)

# Extract detected project
detected_project = content_gpt.get('project', 'Unknown')

# Prepare template variables
now = datetime.now()
if date_str is None:
    date_str = now.strftime('%Y-%m-%d')

template_vars = {
    'date': date_str,
    'project_name': detected_project,
    'audio_filename': audio_filename,
    'timestamp': now.strftime('%Y-%m-%d %H:%M:%S'),
    'summary': content_gpt['summary'],
    'completed': content_gpt['completed'],
    'blockers': content_gpt['blockers'],
    'next_steps': content_gpt['next_steps'],
    'thoughts': content_gpt['thoughts'],
    'transcript_link': ""  # Default empty
}

# Save transcript if enabled
if config.save_transcript:
    transcript_path = note_generator_gpt._save_transcript(
        transcript_data['text'],
        date_str,
        detected_project,
        output_path
    )
    # Create relative path for the link
    relative_path = transcript_path.name if transcript_path.parent == output_path else f"{config.transcript_folder}/{transcript_path.name}"
    template_vars['transcript_link'] = f"## 📝 Full Transcript\n[View complete transcript]({relative_path})\n"

# Fill template
note_content = note_generator_gpt.get_daily_note_template().format(**template_vars)

# Create output file
daily_note_path = output_path / f"{date_str}_{detected_project}_gpt_debug.md"

# Handle existing file
if daily_note_path.exists():
    timestamp_suffix = now.strftime('%H%M%S')
    daily_note_path = output_path / f"{date_str}_{detected_project}_{timestamp_suffix}.md"
    print(f"Daily note exists, creating: {daily_note_path.name}")

# Write file
with open(daily_note_path, 'w', encoding='utf-8') as f:
    f.write(note_content)
    
print(f"Created daily note for project '{detected_project}': {daily_note_path}")

# Extract and add todo items
print("Checking for todo items in transcript...")
todo_items = note_generator_gpt.todo_manager.extract_todos(transcript_data['text'], detected_project)

if todo_items:
    print(f"Found {len(todo_items)} todo items.")
    note_generator_gpt.todo_manager.add_todos_to_project(detected_project, todo_items, date_str)
else:
    print("No todo items found in transcript.")



Saved transcript: c:\Users\nicco\OneDrive\Documenti\Obsidian\Vault\1. Projects\Daily Notes\transcripts\2025-06-25_Saliency_transcript_145242.md
Created daily note for project 'Saliency': c:\Users\nicco\OneDrive\Documenti\Obsidian\Vault\1. Projects\Daily Notes\2025-06-25_Saliency_gpt_debug.md
Checking for todo items in transcript...
No todo items found in transcript.


In [8]:
content_gpt

{'project': 'Saliency',
 'summary': "Today, I focused primarily on the Saliency project, enhancing the plotting functions used to inspect data from diffusion runs. I adapted the plotting functions to work with the updated data format, where the diffusion model's sampling function now returns a sampled sequence alongside a dictionary containing stacked tensors of probabilities for all steps, instead of a list of tensors per step. This required significant adjustments, which are now successfully implemented. Additionally, I added smaller features such as printing metadata on the interactive plots and made various improvements to the visualization and data-saving scripts. I prepared and submitted a pull request (PR) for these changes and reviewed a PR submitted by Ishan. Lastly, I continued updating the conference post by adding comments on plots presented last Thursday.",
 'completed': '- Adapted plotting functions to handle new data format with stacked tensors of probabilities for all d

In [9]:
response_gpt.choices[0].message.content

'{\n  "project": "Saliency",\n  "summary": "Today, I focused primarily on the Saliency project, enhancing the plotting functions used to inspect data from diffusion runs. I adapted the plotting functions to work with the updated data format, where the diffusion model\'s sampling function now returns a sampled sequence alongside a dictionary containing stacked tensors of probabilities for all steps, instead of a list of tensors per step. This required significant adjustments, which are now successfully implemented. Additionally, I added smaller features such as printing metadata on the interactive plots and made various improvements to the visualization and data-saving scripts. I prepared and submitted a pull request (PR) for these changes and reviewed a PR submitted by Ishan. Lastly, I continued updating the conference post by adding comments on plots presented last Thursday.",\n  "completed": "- Adapted plotting functions to handle new data format with stacked tensors of probabilities

In [10]:
audio_filename=audio_files[0].name
output_path=config.daily_notes_path

# Generate content from transcript
content_deep, response_deep = note_generator_deep.generate_note_content(transcript_data['text'], available_projects)

# Extract detected project
detected_project = content_deep.get('project', 'Unknown')

# Prepare template variables
now = datetime.now()
if date_str is None:
    date_str = now.strftime('%Y-%m-%d')

template_vars = {
    'date': date_str,
    'project_name': detected_project,
    'audio_filename': audio_filename,
    'timestamp': now.strftime('%Y-%m-%d %H:%M:%S'),
    'summary': content_deep['summary'],
    'completed': content_deep['completed'],
    'blockers': content_deep['blockers'],
    'next_steps': content_deep['next_steps'],
    'thoughts': content_deep['thoughts'],
    'transcript_link': ""  # Default empty
}

# Save transcript if enabled
if config.save_transcript:
    transcript_path = note_generator_deep._save_transcript(
        transcript_data['text'],
        date_str,
        detected_project,
        output_path
    )
    # Create relative path for the link
    relative_path = transcript_path.name if transcript_path.parent == output_path else f"{config.transcript_folder}/{transcript_path.name}"
    template_vars['transcript_link'] = f"## 📝 Full Transcript\n[View complete transcript]({relative_path})\n"

# Fill template
note_content = note_generator_deep.get_daily_note_template().format(**template_vars)

# Create output file
daily_note_path = output_path / f"{date_str}_{detected_project}_deep_debug.md"

# Handle existing file
if daily_note_path.exists():
    timestamp_suffix = now.strftime('%H%M%S')
    daily_note_path = output_path / f"{date_str}_{detected_project}_{timestamp_suffix}.md"
    print(f"Daily note exists, creating: {daily_note_path.name}")

# Write file
with open(daily_note_path, 'w', encoding='utf-8') as f:
    f.write(note_content)
    
print(f"Created daily note for project '{detected_project}': {daily_note_path}")

# Extract and add todo items
print("Checking for todo items in transcript...")
todo_items = note_generator_deep.todo_manager.extract_todos(transcript_data['text'], detected_project)

if todo_items:
    print(f"Found {len(todo_items)} todo items.")
    note_generator_deep.todo_manager.add_todos_to_project(detected_project, todo_items, date_str)
else:
    print("No todo items found in transcript.")

Fallback: extracting content manually, JSON parsing failed
Saved transcript: c:\Users\nicco\OneDrive\Documenti\Obsidian\Vault\1. Projects\Daily Notes\transcripts\2025-06-25_Saliency_transcript_145302.md
Created daily note for project 'Saliency': c:\Users\nicco\OneDrive\Documenti\Obsidian\Vault\1. Projects\Daily Notes\2025-06-25_Saliency_deep_debug.md
Checking for todo items in transcript...
No todo items found in transcript.


In [11]:
content_deep

{'project': 'Saliency',
 'summary': "Today's work focused on the Saliency project, primarily improving plotting functions for inspecting diffusion model data. Key achievements include making the plotting functions work with the new data structure (stacked tensors instead of lists), adding metadata printing for interactive plots, and preparing a PR. Also reviewed a colleague's PR and continued updating a conference post with plot comments.",
 'completed': "- Made plotting functions work with stacked tensor data structure (instead of list of tensors)  \n- Added metadata printing feature for interactive plots  \n- Made small changes to visualization/saving scripts  \n- Prepared and submitted a PR  \n- Reviewed Ishan's PR",
 'blockers': 'None mentioned',
 'next_steps': 'None mentioned',
 'thoughts': '- Conference post updates: Writing comments on plots shown last Thursday'}

In [12]:
response_deep.choices[0].message.content

'{\n    "project": "Saliency",\n    "summary": "Today\'s work focused on the Saliency project, primarily improving plotting functions for inspecting diffusion model data. Key achievements include making the plotting functions work with the new data structure (stacked tensors instead of lists), adding metadata printing for interactive plots, and preparing a PR. Also reviewed a colleague\'s PR and continued updating a conference post with plot comments.",\n    "completed": "- Made plotting functions work with stacked tensor data structure (instead of list of tensors)  \n- Added metadata printing feature for interactive plots  \n- Made small changes to visualization/saving scripts  \n- Prepared and submitted a PR  \n- Reviewed Ishan\'s PR",\n    "blockers": "None mentioned",\n    "next_steps": "None mentioned",\n    "thoughts": "- Conference post updates: Writing comments on plots shown last Thursday"\n}'

In [3]:
supported_formats = config.config_data['audio']['supported_formats']
audio_files = []

for format_ext in supported_formats:
    audio_files.extend(config.audio_input_path.glob(f"*{format_ext}"))
    
sorted(audio_files)
audio_files

[WindowsPath('c:/Users/nicco/OneDrive/Documenti/Obsidian/daily_notes/AudioInbox/test_record.m4a')]

In [4]:
audio_path = audio_files[0]

# 1. Transcribe audio
transcript_data = audio_processor.transcribe(audio_path)
print(f"Transcription completed ({len(transcript_data['text'])} chars)")


Transcribing: test_record.m4a
✓ Audio validation passed: Valid audio file: 155.0s
Normalizing audio for optimal transcription...
✓ Audio normalized
Starting transcription...


100%|██████████| 6/6 [00:58<00:00,  9.83s/it]

✓ Transcription completed
Transcription completed (1241 chars)


In [8]:
# 2. Get available projects
available_projects = config.get_available_projects()
print(f"Available projects: {', '.join(available_projects)}")

# 3. Generate daily note with project detection
note_path = note_generator.create_daily_note(
    transcript_data=transcript_data,
    available_projects=available_projects,
    audio_filename=audio_path.name,
    output_path=config.daily_notes_path
)

# 4. Delete audio file if configured
if config.config_data['audio']['delete_after_processing']:
    success = audio_processor.delete_audio_file(audio_path)
    if not success:
        print(f"⚠ Warning: Could not delete {audio_path.name}")

print(f"✅ Success! Note: {note_path.name}")

Available projects: ENPICOM, Saliency
ChatCompletion(id='chatcmpl-BeFtknkoH2DEpzvmZUkzjzkaKR9Jl', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "project": "Saliency",\n  "summary": "- Today I worked on my main project, Palency, which I believe is a reference to Saliency.\\n- I continued conducting experiments to test the gradient flow on the prediction model over the input embeddings.\\n- I focused on debugging an issue from Friday that was causing zero gradient in certain regions of the antibody sequences.\\n- The problem was related to how sequences were fragmented by the anarchy sequence segment function.\\n- I discovered that some sequences weren\'t properly segmented, particularly in the framework region 4.\\n- This segmentation issue was affecting the indexing and lists, leading to inaccurate results.\\n- I made adjustments to make the process resistant to incorrectly fragmented sequences.\\n- Despite the fixes,

## Interactvie

In [2]:
processor = DailyNotesProcessor('config.yaml')

Loading Whisper model 'medium'...
✓ Whisper model loaded successfully
Setup complete. Drop audio files in: c:\Users\nicco\OneDrive\Documenti\Obsidian\daily_notes\AudioInbox


In [3]:
processor.run_interactive() 

🎯 Daily Notes Processor - Interactive Mode

Choose an option:
1. 📁 Scan for new audio files
2. 🎤 Record new voice note
3. ⚙️  Configure audio device
4. 📋 Show current settings
5. 📅 Generate timeline
6. 🚪 Exit

🎤 Recording Voice Note
------------------------------

🎙️  Ready to record to: c:\Users\nicco\OneDrive\Documenti\Obsidian\daily_notes\AudioInbox
🔴 Recording started using: System Default (Microphone (2- EDIFIER W830NB))
Press SPACE to stop or Ctrl+C to cancel...

⏹️  Stopping recording...
✅ Recording stopped! Duration: 245.18 seconds
✅ Audio saved: voice_note_20250603_172430.wav (21118.0 KB)

✅ Recording saved: voice_note_20250603_172430.wav

🔄 Processing voice_note_20250603_172430.wav...

Processing: voice_note_20250603_172430.wav
Transcribing: voice_note_20250603_172430.wav
✓ Audio validation passed: Valid audio file: 245.2s
Normalizing audio for optimal transcription...
✓ Audio normalized
Starting transcription...


c:\Users\nicco\OneDrive\Documenti\Obsidian\.venv\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Detected language: English


100%|██████████| 24517/24517 [02:40<00:00, 152.40frames/s]


✓ Transcription completed
Transcription completed (2239 chars)
Available projects: ENPICOM, Saliency
Daily note exists, creating: 2025-06-03_Saliency_173143.md
Created daily note for project 'Saliency': c:\Users\nicco\OneDrive\Documenti\Obsidian\Vault\1. Projects\Daily Notes\2025-06-03_Saliency_173143.md
✅ Success! Note: 2025-06-03_Saliency_173143.md
✅ Voice note processed successfully!

Choose an option:
1. 📁 Scan for new audio files
2. 🎤 Record new voice note
3. ⚙️  Configure audio device
4. 📋 Show current settings
5. 📅 Generate timeline
6. 🚪 Exit

🎤 Recording Voice Note
------------------------------

🎙️  Ready to record to: c:\Users\nicco\OneDrive\Documenti\Obsidian\daily_notes\AudioInbox
🔴 Recording started using: System Default (Microphone (2- EDIFIER W830NB))
Press SPACE to stop or Ctrl+C to cancel...

⏹️  Stopping recording...
✅ Recording stopped! Duration: 203.06 seconds
✅ Audio saved: voice_note_20250603_173505.wav (17490.0 KB)

✅ Recording saved: voice_note_20250603_173505.w

100%|██████████| 20305/20305 [01:27<00:00, 232.37frames/s]


✓ Transcription completed
Transcription completed (1519 chars)
Available projects: ENPICOM, Saliency
Created daily note for project 'Saliency': c:\Users\nicco\OneDrive\Documenti\Obsidian\Vault\1. Projects\Daily Notes\2025-06-03_Saliency.md
✅ Success! Note: 2025-06-03_Saliency.md
✅ Voice note processed successfully!

Choose an option:
1. 📁 Scan for new audio files
2. 🎤 Record new voice note
3. ⚙️  Configure audio device
4. 📋 Show current settings
5. 📅 Generate timeline
6. 🚪 Exit

📋 Current Settings
------------------------------
Projects Path: c:\Users\nicco\OneDrive\Documenti\Obsidian\Vault\1. Projects
Audio Inbox: c:\Users\nicco\OneDrive\Documenti\Obsidian\daily_notes\AudioInbox
Daily Notes: c:\Users\nicco\OneDrive\Documenti\Obsidian\Vault\1. Projects\Daily Notes
Delete after processing: False
Audio Device: System Default (Microphone (2- EDIFIER W830NB))
Available Projects: ENPICOM, Saliency

Choose an option:
1. 📁 Scan for new audio files
2. 🎤 Record new voice note
3. ⚙️  Configure a

In [ ]:
print(f"\n🎤 Recording Voice Note") 
print("-" * 30)

# Test default device first
if not processor.audio_recorder.test_default_device():
    print("⚠️  Default audio device not available.")
    device_id = processor.audio_recorder.select_device()
    processor.audio_recorder.selected_device_id = device_id


🎤 Recording Voice Note
------------------------------
